In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [2]:
#local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\2599.pdf"
local_path = r"D:\GitHub\Projetos\Mestrado\EnergyContext\pdf\appendixa_0.pdf"

# Local PDF file uploads
if local_path:
  loader = UnstructuredPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [4]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [5]:
# Add to vector database
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 3/3 [00:11<00:00,  3.89s/it]


In [6]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [7]:
# LLM from Ollama
local_model = "mistral"
llm = ChatOllama(model=local_model)

In [8]:
'''QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)'''
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database, and only the database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [9]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
chain.invoke("What are technologies used to provide indoor environmental comfort?")

OllamaEmbeddings: 100%|██████████| 1/1 [00:04<00:00,  4.41s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.08s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.05s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.09s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.24s/it]
Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


' The provided text does not explicitly mention the technologies used to provide indoor environmental comfort, but it does discuss policies related to energy efficiency and renewable energy, which can contribute to improving indoor environmental quality by reducing emissions from fossil fuel-fired electric generating units and improving air quality. Some examples of technologies that can provide indoor environmental comfort are:\n\n1. Heating, Ventilation, and Air Conditioning (HVAC) systems - These systems control the temperature, humidity, and air quality in buildings by circulating air throughout the space, removing pollutants and contaminants, and providing heating or cooling as needed.\n2. Energy-efficient lighting - LED lights are a popular choice for energy-efficient lighting solutions that reduce energy consumption and can improve indoor lighting quality.\n3. Insulation materials - Proper insulation in buildings can help maintain comfortable temperatures, reduce heat loss or ga

In [12]:
from transformers import pipeline

# Load a question generation pipeline
question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qg-hl")

# Generate synthetic questions and answers
synthetic_data = []
for chunk in chunks:
    # Assuming the text content is accessed via the 'page_content' attribute
    text = chunk.page_content
    question = question_generator("generate question: " + text)[0]['generated_text']
    synthetic_data.append((question, text))

# Separate into queries and expected answers
test_queries, expected_answers = zip(*synthetic_data)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Token indices sequence length is longer than the specified maximum sequence length for this model (1552 > 512). Running this sequence through the model will result in indexing errors
d:\ProgramData\anaconda3\Lib\site-packages\transformers\generation\utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the max

In [13]:
from datasets import load_metric
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load the metrics
bleu_metric = load_metric("bleu")
rouge_metric = load_metric("rouge")

# Load a sentence transformer model for semantic similarity
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to evaluate the RAG system using BLEU, ROUGE, and semantic similarity
def evaluate_rag_system(chain, test_queries, expected_answers):
    predictions = [chain.invoke(query) for query in test_queries]
    print("Predictions:", predictions)  # Debug: Print predictions to see what the system returns

    # Initialize lists to hold references and predictions for metrics calculation
    references = [[ref.split()] for ref in expected_answers]  # BLEU expects list of lists of tokens
    preds = [pred.split() for pred in predictions]

    # Calculate BLEU score
    bleu_metric.add_batch(predictions=preds, references=references)
    bleu_score = bleu_metric.compute()['bleu']

    # Calculate ROUGE scores
    rouge_metric.add_batch(predictions=predictions, references=expected_answers)
    rouge_scores = rouge_metric.compute()
    
    # Calculate semantic similarity
    semantic_similarities = []
    for pred, ref in zip(predictions, expected_answers):
        pred_embedding = model.encode(pred, convert_to_tensor=True)
        ref_embedding = model.encode(ref, convert_to_tensor=True)
        semantic_similarity = util.pytorch_cos_sim(pred_embedding, ref_embedding).item()
        semantic_similarities.append(semantic_similarity)
    
    avg_semantic_similarity = np.mean(semantic_similarities)

    # Print results
    print(f"BLEU Score: {bleu_score:.2f}")
    print(f"ROUGE-1 Score: {rouge_scores['rouge1'].mid.fmeasure:.2f}")
    print(f"ROUGE-2 Score: {rouge_scores['rouge2'].mid.fmeasure:.2f}")
    print(f"ROUGE-L Score: {rouge_scores['rougeL'].mid.fmeasure:.2f}")
    print(f"Average Semantic Similarity: {avg_semantic_similarity:.2f}")

    return bleu_score, rouge_scores, avg_semantic_similarity

# Evaluate the RAG system
evaluate_rag_system(chain, test_queries, expected_answers)


C:\Users\Matheus\AppData\Local\Temp\ipykernel_18412\1570586788.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")
d:\ProgramData\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
d:\ProgramData\anaconda3\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metr

Predictions: [' The term you are looking for isn\'t explicitly stated in the provided text, but it seems to refer to a combination of "State Implementation Plan (SIP)" and "Tribal Implementation Plan (TIP)". These are plans that states and tribes submit to the U.S. Environmental Protection Agency (EPA) for review, which can include energy efficiency policies and programs. However, it\'s not mentioned as a specific road map for incorporating energy efficiency in these plans. If I missed something, please let me know!', ' The document does not specify a specific organization that oversees the EE (Energy Efficiency) program, but it does mention the North American Electric Reliability Corporation (NERC) which ensures the reliability of the North American bulk power system. However, this is related to electric sector and not specifically Energy Efficiency programs.', ' The term you\'re looking for is "portfolio" or "program". These terms can refer to a collection of energy efficiency projec

(7.415305365559244e-07,
 {'rouge1': AggregateScore(low=Score(precision=0.6395348837209303, recall=0.035422343324250684, fmeasure=0.06776715899218072), mid=Score(precision=0.6980778929672177, recall=0.05392628003744809, fmeasure=0.09910564961118339), high=Score(precision=0.7799999999999999, recall=0.07617728531855955, fmeasure=0.13613861386138612)),
  'rouge2': AggregateScore(low=Score(precision=0.18823529411764706, recall=0.01818181818181818, fmeasure=0.034812880765883375), mid=Score(precision=0.29229545476727276, recall=0.020333673901526222, fmeasure=0.03764817362232107), high=Score(precision=0.40816326530612246, recall=0.022191400832177532, fmeasure=0.03970223325062035)),
  'rougeL': AggregateScore(low=Score(precision=0.36046511627906974, recall=0.02633969118982743, fmeasure=0.05039096437880105), mid=Score(precision=0.4701148781171196, recall=0.034740738612536626, fmeasure=0.06405928316311015), high=Score(precision=0.58, recall=0.04293628808864266, fmeasure=0.07673267326732673)),
  '